Выполнила Леопаева Л.А., 6402-010302D

Лабораторная работа 5

In [1]:
import numpy as np
from itertools import combinations, product
from statistics import mode

5.1 Функция формирования порождающей матрицы кода РидаМаллера в каноническом виде.

In [2]:
def generate_binary_vectors(cols):
    return [list(c)[::-1] for c in product([0, 1], repeat=cols)]


In [3]:
def check_vector(indexes, u):
    return int(len(indexes) == 0 or np.all(np.asarray(u)[indexes] == 0))

In [4]:
def vectorize_function(indexes, size):
    return [check_vector(indexes, b) for b in generate_binary_vectors(size)]

In [5]:
def reed_muller_G(r, m):
    return np.array([vectorize_function(idx, m) for idx in generate_all_indexes(r, m)])

In [6]:
def reed_muller_H(index, m):
    return [u for u in generate_binary_vectors(m) if check_vector(index, u) == 1]

In [7]:
def get_complementary_indices(indexes, m):
    return [i for i in range(m) if i not in indexes]

In [8]:
def get_combinations(size, m):
    return [list(comb) for comb in combinations(range(m - 1, -1, -1), size)]

In [9]:
def generate_all_indexes(r, m):
    index_array = []
    for i in range(r + 1):
        index_array.extend(get_combinations(i, m))
    return index_array

In [10]:
def f_with_t(index, t, m):
    return [int(np.array_equal(np.asarray(b)[index], np.asarray(t)[index])) for b in generate_binary_vectors(m)]

5.2. Алгоритм мажоритарного декодирования для кода РидаМаллера

In [11]:
def majority_vote_decoding(w, H, idx, m):
    complementary = get_complementary_indices(idx, m)
    vote_vectors = [f_with_t(complementary, u, m) for u in H]
    votes = [np.dot(np.asarray(v), np.asarray(w)) % 2 for v in vote_vectors]
    return mode(votes)

In [12]:
def decode_with_reed_muller(word_with_error, r, m, G):
    a = np.zeros((G.shape[0]), dtype=int)
    all_indexes = generate_all_indexes(r, m)

    for step in range(r, -1, -1):
        index_combinations = get_combinations(step, m)
        first_step_results = []

        for idx in index_combinations:
            H = reed_muller_H(idx, m)
            first_step_results.append(majority_vote_decoding(word_with_error, H, idx, m))
        position = all_indexes.index(index_combinations[0])

        for i in range(len(first_step_results)):
            a[i + position] = first_step_results[i]

        if step != 0:
            word_with_error = (a.T @ G + word_with_error) % 2
        else:
            word_with_error = a.T @ G % 2
        print(f'Слово после шага {abs(step - r - 1)}: {word_with_error}')

    return word_with_error

5.3. Экспериментальная проверка алгоритма декодирования

In [13]:
def reed_muller_experiment():
    r, m = 2, 4
    G = reed_muller_G(r, m)
    print('Порождающая матрица G: \n', G)
    original_word = np.array([0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0])
    print('Исходное слово: ', original_word)
    encrypted_word = original_word @ G % 2
    print('Зашифрованное слово: ', encrypted_word)

    E = np.eye(16, dtype=int)
    word_with_error = (encrypted_word + E[4]) % 2
    print('Слово с ошибкой: ', word_with_error)

    decoded_word = decode_with_reed_muller(word_with_error, r, m, G)
    print('Исправленное сообщение: ', decoded_word)

    if np.array_equal(encrypted_word, decoded_word):
        print("Отправленное слово и декодированное совпадают.\n")
    else:
        print("Отправленное слово и декодированное не совпадают.\n")

In [14]:
reed_muller_experiment()

Порождающая матрица G: 
 [[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0]
 [1 1 1 1 0 0 0 0 1 1 1 1 0 0 0 0]
 [1 1 0 0 1 1 0 0 1 1 0 0 1 1 0 0]
 [1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0]
 [1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0]
 [1 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0]
 [1 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0]
 [1 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0]
 [1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0]]
Исходное слово:  [0 1 0 0 0 0 0 0 0 1 0]
Зашифрованное слово:  [0 1 0 1 1 1 1 1 1 0 1 0 0 0 0 0]
Слово с ошибкой:  [0 1 0 1 0 1 1 1 1 0 1 0 0 0 0 0]
Слово после шага 1: [1 1 1 1 0 1 1 1 0 0 0 0 0 0 0 0]
Слово после шага 2: [1 0 1 0 1 0 0 0 1 0 1 0 0 0 0 0]
Слово после шага 3: [0 1 0 1 1 1 1 1 1 0 1 0 0 0 0 0]
Исправленное сообщение:  [0 1 0 1 1 1 1 1 1 0 1 0 0 0 0 0]
Отправленное слово и декодированное совпадают.

